In [1]:
pip install scikit-learn

In [2]:
pip install --user -U nltk

In [3]:
pip install newsapi-python

In [4]:
pip install pandas

In [5]:
from newsapi import NewsApiClient

import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
newsapi = NewsApiClient(api_key='427e3e2dd1be49ebbfa15f34fbbac059')
#use your API key here

In [7]:
tech_articles = newsapi.get_everything(q='tech', language='en', \
page_size=100)
tech_articles

{'status': 'ok',
 'totalResults': 29112,
 'articles': [{'source': {'id': 'wired', 'name': 'Wired'},
   'author': 'Gabriel Leão',
   'title': "Brazilian Gamers Aren't Waiting for Hollywood. They're Building Their Own Cons",
   'description': 'Perifacon shows the power of gaming, geek culture, and tech outside of uptown São Paulo.',
   'url': 'https://www.wired.com/story/brazil-games-fandom-perifacon/',
   'urlToImage': 'https://media.wired.com/photos/658399bd032a5a539d43fb81/191:100/w_1280,c_limit/Perifacon-2023-Culture-IMG_6731.jpg',
   'publishedAt': '2023-12-27T14:00:00Z',
   'content': 'In 2022, I interviewed journalistMariana Ayrez, who opened my eyes to the relevance of Perifacon. She reiterated that while other events promote pop culture and bring together artists and the public,… [+2580 chars]'},
  {'source': {'id': 'engadget', 'name': 'Engadget'},
   'author': 'Karissa Bell,Pranav Dixit',
   'title': 'How the EU forced tech companies to change in 2023',
   'description': 'This 

In [8]:
tech_articles.keys()

dict_keys(['status', 'totalResults', 'articles'])

In [9]:
import pandas as pd

tech = pd.DataFrame(tech_articles['articles'])
tech

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'wired', 'name': 'Wired'}",Gabriel Leão,Brazilian Gamers Aren't Waiting for Hollywood....,"Perifacon shows the power of gaming, geek cult...",https://www.wired.com/story/brazil-games-fando...,https://media.wired.com/photos/658399bd032a5a5...,2023-12-27T14:00:00Z,"In 2022, I interviewed journalistMariana Ayrez..."
1,"{'id': 'engadget', 'name': 'Engadget'}","Karissa Bell,Pranav Dixit",How the EU forced tech companies to change in ...,"This year, tech companies have made concession...",https://www.engadget.com/how-the-eu-forced-tec...,https://s.yimg.com/ny/api/res/1.2/beleWjXfooZ6...,2023-12-26T15:30:23Z,"This year, tech companies have made concession..."
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Mat Smith,The Morning After: The biggest tech stories of...,What do you do when the tech world mostly grin...,https://consent.yahoo.com/v2/collectConsent?se...,None,2023-12-30T14:00:22Z,"Si vous cliquez sur « Tout accepter », nos par..."
3,"{'id': 'wired', 'name': 'Wired'}",Brendan Nystedt,Analogue Duo Review: A Return to PC Engine and...,The Analogue Duo puts all your PC Engine and T...,https://www.wired.com/review/analogue-duo/,https://media.wired.com/photos/6577258a79c88ff...,2023-12-11T16:00:00Z,A big hurdle to enjoying these games today is ...
4,"{'id': 'wired', 'name': 'Wired'}",Morgan Meaker,What Happens When Facebook Heats Your Home,Big Tech data centers are not only being used ...,https://www.wired.com/story/facebook-energy-he...,https://media.wired.com/photos/658226459e524a9...,2023-12-20T12:00:00Z,"For Big Tech, there are few better places to e..."
...,...,...,...,...,...,...,...,...
95,"{'id': 'business-insider', 'name': 'Business I...",Julia Naftulin,An escort and stripper who caters to Wall Stre...,Mia Lee entertains bankers and tech executives...,https://www.businessinsider.com/ex-wall-street...,https://i.insider.com/657b7b040ec98e92f74fb461...,2023-12-18T11:28:01Z,Penelope Dario; iStock; Rebecca Zisser/BI\r\n<...
96,"{'id': 'business-insider', 'name': 'Business I...",kteo@insider.com (Kai Xiang Teo),The salaries of Wikimedia executives are spark...,"""A mid career software engineer at FB makes mo...",https://www.businessinsider.com/wikipedia-wiki...,https://i.insider.com/657a72f27a3c8094d5dcd10b...,2023-12-14T10:10:34Z,Wikimedia's executive salaries are the talk of...
97,"{'id': 'the-verge', 'name': 'The Verge'}",Tom Warren,Microsoft’s Copilot is getting OpenAI’s latest...,Microsoft is promising even more AI improvemen...,https://www.theverge.com/2023/12/5/23989052/mi...,https://cdn.vox-cdn.com/thumbor/nwvkfa-cTgy4PU...,2023-12-05T15:01:30Z,Microsofts Copilot is getting OpenAIs latest m...
98,"{'id': 'the-verge', 'name': 'The Verge'}",Sheena Vasani,The second season of The Last of Us will debut...,In a teaser offering a preview of its 2024 lin...,https://www.theverge.com/2023/12/6/23991040/th...,https://cdn.vox-cdn.com/thumbor/1hvxVnIbmTmLhM...,2023-12-06T22:15:12Z,The second season of The Last of Us will debut...


In [28]:
tech.shape

(100, 9)

In [29]:
tech.describe()

,source,author,title,description,url,urlToImage,publishedAt,content,category
count,100,100,100,100,100,98,100,100,100
unique,17,78,100,100,100,98,100,100,1
top,"{'id': 'business-insider', 'name': 'Business I...",Wes Davis,Brazilian Gamers Aren't Waiting for Hollywood....,"Perifacon shows the power of gaming, geek cult...",https://www.wired.com/story/brazil-games-fando...,https://media.wired.com/photos/658399bd032a5a5...,2023-12-27T14:00:00Z,"In 2022, I interviewed journalistMariana Ayrez...",Tech
freq,30,6,1,1,1,1,1,1,100


In [30]:
tech.columns

Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content', 'category'],
      dtype='object')

In [31]:
tech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source       100 non-null    object
 1   author       100 non-null    object
 2   title        100 non-null    object
 3   description  100 non-null    object
 4   url          100 non-null    object
 5   urlToImage   98 non-null     object
 6   publishedAt  100 non-null    object
 7   content      100 non-null    object
 8   category     100 non-null    object
dtypes: object(9)
memory usage: 7.2+ KB


In [10]:
#adding the tech category
tech['category'] = 'Tech'
tech

,source,author,title,description,url,urlToImage,publishedAt,content,category
0,"{'id': 'wired', 'name': 'Wired'}",Gabriel Leão,Brazilian Gamers Aren't Waiting for Hollywood....,"Perifacon shows the power of gaming, geek cult...",https://www.wired.com/story/brazil-games-fando...,https://media.wired.com/photos/658399bd032a5a5...,2023-12-27T14:00:00Z,"In 2022, I interviewed journalistMariana Ayrez...",Tech
1,"{'id': 'engadget', 'name': 'Engadget'}","Karissa Bell,Pranav Dixit",How the EU forced tech companies to change in ...,"This year, tech companies have made concession...",https://www.engadget.com/how-the-eu-forced-tec...,https://s.yimg.com/ny/api/res/1.2/beleWjXfooZ6...,2023-12-26T15:30:23Z,"This year, tech companies have made concession...",Tech
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Mat Smith,The Morning After: The biggest tech stories of...,What do you do when the tech world mostly grin...,https://consent.yahoo.com/v2/collectConsent?se...,None,2023-12-30T14:00:22Z,"Si vous cliquez sur « Tout accepter », nos par...",Tech
3,"{'id': 'wired', 'name': 'Wired'}",Brendan Nystedt,Analogue Duo Review: A Return to PC Engine and...,The Analogue Duo puts all your PC Engine and T...,https://www.wired.com/review/analogue-duo/,https://media.wired.com/photos/6577258a79c88ff...,2023-12-11T16:00:00Z,A big hurdle to enjoying these games today is ...,Tech
4,"{'id': 'wired', 'name': 'Wired'}",Morgan Meaker,What Happens When Facebook Heats Your Home,Big Tech data centers are not only being used ...,https://www.wired.com/story/facebook-energy-he...,https://media.wired.com/photos/658226459e524a9...,2023-12-20T12:00:00Z,"For Big Tech, there are few better places to e...",Tech
...,...,...,...,...,...,...,...,...,...
95,"{'id': 'business-insider', 'name': 'Business I...",Julia Naftulin,An escort and stripper who caters to Wall Stre...,Mia Lee entertains bankers and tech executives...,https://www.businessinsider.com/ex-wall-street...,https://i.insider.com/657b7b040ec98e92f74fb461...,2023-12-18T11:28:01Z,Penelope Dario; iStock; Rebecca Zisser/BI\r\n<...,Tech
96,"{'id': 'business-insider', 'name': 'Business I...",kteo@insider.com (Kai Xiang Teo),The salaries of Wikimedia executives are spark...,"""A mid career software engineer at FB makes mo...",https://www.businessinsider.com/wikipedia-wiki...,https://i.insider.com/657a72f27a3c8094d5dcd10b...,2023-12-14T10:10:34Z,Wikimedia's executive salaries are the talk of...,Tech
97,"{'id': 'the-verge', 'name': 'The Verge'}",Tom Warren,Microsoft’s Copilot is getting OpenAI’s latest...,Microsoft is promising even more AI improvemen...,https://www.theverge.com/2023/12/5/23989052/mi...,https://cdn.vox-cdn.com/thumbor/nwvkfa-cTgy4PU...,2023-12-05T15:01:30Z,Microsofts Copilot is getting OpenAIs latest m...,Tech
98,"{'id': 'the-verge', 'name': 'The Verge'}",Sheena Vasani,The second season of The Last of Us will debut...,In a teaser offering a preview of its 2024 lin...,https://www.theverge.com/2023/12/6/23991040/th...,https://cdn.vox-cdn.com/thumbor/1hvxVnIbmTmLhM...,2023-12-06T22:15:12Z,The second season of The Last of Us will debut...,Tech


In [11]:
entertainment_articles = newsapi.get_everything(q='entertainment',\
language='en', page_size=100)
business_articles = newsapi.get_everything(q='business',\
language='en', page_size=100)
sports_articles = newsapi.get_everything(q='sports',\
language='en', page_size=100)
politics_articles = newsapi.get_everything(q='politics',\
language='en', page_size=100)
travel_articles = newsapi.get_everything(q='travel',\
language='en', page_size=100)
food_articles = newsapi.get_everything(q='food',\
language='en', page_size=100)
health_articles = newsapi.get_everything(q='health',\
language='en', page_size=100)

In [12]:
entertainment = pd.DataFrame(entertainment_articles['articles'])
entertainment['category'] = 'Entertainment'
business = pd.DataFrame(business_articles['articles'])
business['category'] = 'Business'
sports = pd.DataFrame(sports_articles['articles'])
sports['category'] = 'Sports'
politics = pd.DataFrame(politics_articles['articles'])
politics['category'] = 'Politics'
travel = pd.DataFrame(travel_articles['articles'])
travel['category'] = 'Travel'
food = pd.DataFrame(food_articles['articles'])
food['category'] = 'Food'
health = pd.DataFrame(health_articles['articles'])
health['category'] = 'Health'

In [13]:
categories = [tech, entertainment, business, sports, politics, \
travel, food, health]
df = pd.concat(categories)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 99
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source       800 non-null    object
 1   author       771 non-null    object
 2   title        800 non-null    object
 3   description  785 non-null    object
 4   url          800 non-null    object
 5   urlToImage   767 non-null    object
 6   publishedAt  800 non-null    object
 7   content      800 non-null    object
 8   category     800 non-null    object
dtypes: object(9)
memory usage: 62.5+ KB


In [14]:
def cleaned_desc_column(text):
    # Remove commas
    text = re.sub(r',', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Remove full stops
    text = re.sub(r'\.', '', text)
    # Remove single quotes and double quotes
    text = re.sub(r"['\"]", '', text)
    # Remove other non-word characters
    text = re.sub(r'\W', ' ', text)

    text_token = word_tokenize(text)
    stop_words = set(stopwords.words('english'))

    filtered_text = []

    for sw in text_token:
      if sw not in stop_words:
          filtered_text.append(sw)

    text = " ".join(filtered_text)
    return text

In [15]:
df['news_title'] = df['title'].apply(cleaned_desc_column)
df
# The cleaned column 'news_title' is added to the data frame.

,source,author,title,description,url,urlToImage,publishedAt,content,category,news_title
0,"{'id': 'wired', 'name': 'Wired'}",Gabriel Leão,Brazilian Gamers Aren't Waiting for Hollywood....,"Perifacon shows the power of gaming, geek cult...",https://www.wired.com/story/brazil-games-fando...,https://media.wired.com/photos/658399bd032a5a5...,2023-12-27T14:00:00Z,"In 2022, I interviewed journalistMariana Ayrez...",Tech,Brazilian Gamers Arent Waiting Hollywood Theyr...
1,"{'id': 'engadget', 'name': 'Engadget'}","Karissa Bell,Pranav Dixit",How the EU forced tech companies to change in ...,"This year, tech companies have made concession...",https://www.engadget.com/how-the-eu-forced-tec...,https://s.yimg.com/ny/api/res/1.2/beleWjXfooZ6...,2023-12-26T15:30:23Z,"This year, tech companies have made concession...",Tech,How EU forced tech companies change 2023
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Mat Smith,The Morning After: The biggest tech stories of...,What do you do when the tech world mostly grin...,https://consent.yahoo.com/v2/collectConsent?se...,None,2023-12-30T14:00:22Z,"Si vous cliquez sur « Tout accepter », nos par...",Tech,The Morning After The biggest tech stories 2023
3,"{'id': 'wired', 'name': 'Wired'}",Brendan Nystedt,Analogue Duo Review: A Return to PC Engine and...,The Analogue Duo puts all your PC Engine and T...,https://www.wired.com/review/analogue-duo/,https://media.wired.com/photos/6577258a79c88ff...,2023-12-11T16:00:00Z,A big hurdle to enjoying these games today is ...,Tech,Analogue Duo Review A Return PC Engine TurboGr...
4,"{'id': 'wired', 'name': 'Wired'}",Morgan Meaker,What Happens When Facebook Heats Your Home,Big Tech data centers are not only being used ...,https://www.wired.com/story/facebook-energy-he...,https://media.wired.com/photos/658226459e524a9...,2023-12-20T12:00:00Z,"For Big Tech, there are few better places to e...",Tech,What Happens When Facebook Heats Your Home
...,...,...,...,...,...,...,...,...,...,...
95,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Israel-Gaza war: Residents of Khan Younis say ...,Civilians in parts of north and south Gaza hav...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-12-02T18:13:33Z,Israel has carried out intense air strikes on ...,Health,Israel Gaza war Residents Khan Younis say Isra...
96,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,NHS dentistry as we know it 'gone for good',Most adults may need to start paying for care ...,https://www.bbc.co.uk/news/health-67754983,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-12-19T03:42:14Z,The traditional model of NHS dentistry is gone...,Health,NHS dentistry know gone good
97,"{'id': None, 'name': 'NPR'}",Katherine Davis-Young (KJZZ),Supreme Courts in 3 states will hear cases abo...,"Supreme court justices in Arizona, New Mexico,...",https://www.npr.org/2023/12/11/1218357869/stat...,https://media.npr.org/assets/img/2023/12/08/ap...,2023-12-11T10:00:48Z,"Protesters marched in Phoenix, Ariz., when the...",Health,Supreme Courts 3 states hear cases abortion ac...
98,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,"Israel Gaza: What Gaza's reported 20,000 death...",Women and children are majority of those kille...,https://www.bbc.co.uk/news/world-middle-east-6...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-12-20T18:30:08Z,"At least 20,000 Palestinians in Gaza have been...",Health,Israel Gaza What Gazas reported 20000 deaths s...


In [16]:
#getting the category we need for testing
X = df['news_title']
y = df['category']

In [17]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size = 0.30, random_state = 90)
print(X_train.shape)
print(X_test.shape)

(560,)
(240,)


In [21]:
lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(max_iter = 1000)),
              ])

In [22]:
# Train the logistic regression model on the training set
lr.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=1000))])

In [23]:
# Make predictions on the test set
y_pred = lr.predict(X_test)

In [24]:
# Calculate the accuracy of the model
print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")

Accuracy is : 0.49166666666666664


In [25]:
# Test The Model With Different Articles
news = ["Biden to Sign Executive Order That Aims to Make Child Care Cheaper",
       "Google Stock Loses $57 Billion Amid Microsoft’s AI ‘Lead’—And Reports It Could Be Replaced By Bing On Some Smartphones",
       "Poland suspends food imports from Ukraine to assist its farmers",
       "Can AI Solve The Air Traffic Control Problem? Let's Find Out",
       "Woman From Odisha Runs 42.5 KM In UK Marathon Wearing A Saree",
       "Hillary Clinton: Trump cannot win election - but Biden will",
       "Jennifer Aniston and Adam Sandler starrer movie 'Murder Mystery 2' got released on March 24, this year"]

In [26]:
predicted = lr.predict(news)

In [27]:
for doc, category in zip(news, predicted):
     print(category)

Health
Tech
Food
Politics
Food
Politics
Sports
